In [ ]:
import imp
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

from context import rf_pool

In [ ]:
from rf_pool import modules, models, layers, ops
from rf_pool.utils import functions, datasets, stimuli

In [ ]:
imp.reload(functions)
imp.reload(stimuli)
imp.reload(datasets)
imp.reload(ops)
imp.reload(layers)
imp.reload(modules)
imp.reload(models)

In [ ]:
# get MNIST dataset
transform = transforms.ToTensor()
trainset = torchvision.datasets.MNIST(root='../rf_pool/data', train=True, download=True, 
                                       transform=transform)
testset = torchvision.datasets.MNIST(root='../rf_pool/data', train=False, download=True,
                                     transform=transform)

In [ ]:
# create trainloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=2,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=2,
                                         shuffle=True, num_workers=2)

In [ ]:
# initialize deep beleif network
dbn = rf_pool.models.DeepBeliefNetwork()

In [ ]:
# create rf pooling layer
rf_layer = rf_pool.layers.RF_Pool(pool_type='prob', kernel_size=2)

In [ ]:
# add layers 
dbn.append(0, rf_pool.modules.RBM(hidden=torch.nn.Conv2d(1, 24, 11), activation=None, pool=rf_layer, 
                                  vis_activation_fn=torch.nn.Sigmoid(), vis_sample_fn=torch.distributions.Bernoulli,
                                  hid_sample_fn=torch.distributions.Bernoulli))
dbn.append(1, rf_pool.modules.RBM(hidden=torch.nn.Conv2d(24, 40, 9), activation=torch.nn.Sigmoid(), pool=None,
                                  vis_activation_fn=torch.nn.Sigmoid(), vis_sample_fn=torch.distributions.Bernoulli,
                                  hid_sample_fn=torch.distributions.Bernoulli))

In [ ]:
# train first layer with sparsity and lr_scheduler, monitor weights and negative examples
optim = torch.optim.SGD(dbn.layers['0'].parameters(), lr=2e-2, momentum=0.9, weight_decay=2e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optim, step_size=100, gamma=0.99)
loss_history = dbn.train_layer('0', 10, trainloader, optim, k=1, monitor=100,
                               scheduler=scheduler,
                               show_weights={'layer_id': '0', 'cmap': 'gray'}, 
                               show_negative={'layer_id': '0', 'cmap': 'gray'},
                               sparsity={'module_name': None, 'target': 0.002, 'cost': 5.})

In [ ]:
# train second layer, monitor weights and negative examples
# note: second layer weights are projected to the image space
optim = torch.optim.SGD(dbn.layers['1'].parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-4)
dbn.train('1', 1, trainloader, optim, monitor=100, k=1,
          show_weights={'layer_id': '1', 'cmap': 'gray'},
          show_negative={'k': 1, 'cmap': 'gray'})